In [1]:
!pip install langchain langchain-experimental langchain-community langchain-openai openai chromadb pypdf sentence_transformers gradio langchain-together

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 805.5 kB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.1

In [2]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 2.1 MB/s eta 0:00:00


In [3]:
from PyPDF2 import PdfFileReader
from sentence_transformers import SentenceTransformer
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import Together

def setup_rag_model(pdf_path, together_api_key):
    # Load the PDF document
    loader = PyPDFLoader(pdf_path)
    pages = loader.load()

    # Create embeddings
    embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    db = Chroma.from_documents(pages, embedding_function)

    # Initialize the LLM with a conversational model
    llm = Together(
        model="meta-llama/Llama-2-70b-chat-hf",
        max_tokens=256,
        temperature=0.7,  # Adjust temperature for creativity
        top_k=5,  # Increase top_k for diverse responses
        together_api_key=together_api_key
    )

    # Setup the retriever
    retriever = db.as_retriever(similarity_score_threshold=0.8)

    # Define the informal, conversational prompt template for question generation
    question_generation_template = """
    Alright, based on the text below, can you come up with a cool question?

    Here's what we're looking at: {context}
    """

    QUESTION_PROMPT = PromptTemplate(template=f"[INST] {question_generation_template} [/INST]", input_variables=["context"])

    # Define the informal, conversational prompt template for answer generation
    answer_generation_template = """
    Hey, got this question for you! Can you give me a quick answer?

    CONTEXT: {context}
    QUESTION: {question}
    """

    ANSWER_PROMPT = PromptTemplate(template=f"[INST] {answer_generation_template} [/INST]", input_variables=["context", "question"])

    # Create the retrieval chains
    question_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=retriever,
        input_key='query',
        return_source_documents=True,
        chain_type_kwargs={"prompt": QUESTION_PROMPT},
        verbose=True
    )

    answer_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type='stuff',
        retriever=retriever,
        input_key='query',
        return_source_documents=True,
        chain_type_kwargs={"prompt": ANSWER_PROMPT},
        verbose=True
    )

    return question_chain, answer_chain, pages

def generate_question(chain, page_content):
    query = "Hey, can you come up with a fun question based on this?"
    response = chain({"query": query, "context": page_content})
    return response['result']

def answer_question(chain, page_content, question):
    response = chain({"query": question, "context": page_content})
    return response['result']

# Example usage
pdf_path = "/content/somatosensory 1.pdf"  # Replace with the correct path to your PDF document
together_api_key = "5c3aaa83cb623ccf29c3038870387e97d17f64112b00d431f55f6fc19e4d75ab"  # Your Together API key

# Validate the API key
if not together_api_key or together_api_key == "your_together_api_key":
    raise ValueError("Invalid Together API key provided. Please ensure you have a valid key.")

# Setup the RAG model
question_chain, answer_chain, pages = setup_rag_model(pdf_path, together_api_key)

# Loop through each page and perform the conversational Q&A
for page_num, page in enumerate(pages):
    print(f"\n--- Page {page_num + 1} ---")
    page_content = page.page_content

    # Generate a conversational question
    question = generate_question(question_chain, page_content)
    print(f"Generated Question: {question}")

    # Answer the conversational question
    answer = answer_question(answer_chain, page_content, question)
    print(f"Answer: {answer}")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/sett

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `Together` was deprecated in LangChain 0.0.12 and will be removed in 0.3. An updated version of the class exists in the langchain-together package and should be used instead. To use it run `pip install -U langchain-together` and import as `from langchain_together import Together`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(



--- Page 1 ---


> Entering new RetrievalQA chain...

> Finished chain.
Generated Question:   )How do rapidly adapting afferents adjust grip force when lifting objects?

* Can you describe the role of Meissner's corpuscles in recognizing texture?

* What is the difference between rapidly and slowly adapting afferents in terms of response to tactile stimuli?

* How do muscle spindles regulate muscle length and maintain sensitivity?

* What are the different types of joint receptors and their functions?

* How do nociceptors respond to pain, and what is their functional significance?

* What is the significance of the density of Merkel's receptors in the digits and mouth?

* How does activation of Pacinian corpuscles produce a feeling of vibration?

* Can you explain the function of slowly adapting Ruffini corpuscles?

* How do sensor yinformation from Meissner corpuscles and rapidly adapting afferents lead to adjustment of grip force when lifting objects?

* How do nociceptors signal d